In [1]:
# 导入必要的库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from pmdarima.arima import nsdiffs, ndiffs
from pmdarima import auto_arima
import os

In [2]:
file_path = r"C:\Users\huawei\OneDrive\桌面\data.xlsx"

# 读取数据，time列解析为日期但不设为索引
df1 = pd.read_excel(file_path, parse_dates=['time'])

# 按时间列从小到大排序
df1 = df1.sort_values('time').reset_index(drop=True)

print(df1.tail(3))
print(df1.dtypes)
print(f"\n数据形状：{df1.shape}")

         time  ln_gdp_sa  ln_gdp_seasonal_factor  dln_gdp_sa  city_mean_light  \
61 2024-07-01        NaN               -0.056848         NaN        23.956838   
62 2024-10-01        NaN                0.145076         NaN        23.213998   
63 2025-01-01        NaN               -0.077646         NaN        22.387323   

    region_mean_light   ln_city  ln_region       GDP    ln_gdp       pc1  \
61           3.014495  3.176254   1.103432   8632.06  9.063238 -0.217543   
62           2.998758  3.144755   1.098198  10867.59  9.293540 -0.420393   
63           2.792224  3.108495   1.026839   8950.49  9.099464 -0.000561   

         pc2       pc3      dpc2  quarter  dln_city  dln_region  
61 -0.215419  0.043802 -0.025495        3  0.191343    0.241547  
62 -0.176598  0.008674  0.038820        4 -0.031498   -0.005234  
63 -0.158594 -0.012403  0.018004        1 -0.036261   -0.071360  
time                      datetime64[ns]
ln_gdp_sa                        float64
ln_gdp_seasonal_factor  

In [3]:
# 去掉df最后5行
df = df1[:-5]
print(f"删除后数据形状：{df.shape}")

删除后数据形状：(59, 17)


In [4]:
# 1. 数据准备

from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')


target = 'dln_gdp_sa'
base_vars = ['pc1', 'pc3', 'dpc2', 'dln_city', 'dln_region']
max_lag = 4

# 创建滞后变量
data = df.copy()
candidates = []

for var in base_vars + [target]:
    for lag in range(1, max_lag + 1):
        lag_name = f'{var}_lag{lag}'
        data[lag_name] = df[var].shift(lag)
        candidates.append(lag_name)

print(f"候选变量数: {len(candidates)}")
print(f"原始数据: {data.shape}")

# 处理缺失值 - 删除包含缺失值的行
data_clean = data.dropna()
print(f"清理后数据: {data_clean.shape}")
print(f"删除了 {data.shape[0] - data_clean.shape[0]} 行缺失值")

候选变量数: 24
原始数据: (59, 41)
清理后数据: (54, 41)
删除了 5 行缺失值


In [5]:
# Define target variable y
y = data_clean[target]

# Define feature matrix X using only candidate lagged variables
X = data_clean[candidates]

print("Shape of y:", y.shape)
print("Shape of X:", X.shape)

Shape of y: (54,)
Shape of X: (54, 24)


In [6]:
# Define the hyperparameter grid
param_grid = {
    'n_estimators': [75, 100, 125, 150],
    'max_depth': [2, 3, 4, 5]
}

# Define the range of random states
random_states = range(1, 4)

print("Hyperparameter Grid:", param_grid)
print("Random States:", list(random_states))

Hyperparameter Grid: {'n_estimators': [75, 100, 125, 150], 'max_depth': [2, 3, 4, 5]}
Random States: [1, 2, 3]


In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import mean_squared_error
import numpy as np

selected_variables = []
best_models = {}

In [8]:
# Initialize variables for forward selection
selected_variables = []
best_models = {}
results_table = [] # To store results for the final table

max_variables = 7
window_size = 40
forecast_horizon = 5

# Ensure data_clean is sorted by time, as required for time series
data_clean = data_clean.sort_values(by='time').reset_index(drop=True)

# Define the possible candidate variables that haven't been selected yet
remaining_candidates = list(candidates)

print("Starting forward stepwise selection...")

# Forward stepwise selection loop
for i in range(1, max_variables + 1):
    print(f"\nSelecting variable {i}...")
    best_mse_for_step = float('inf')
    best_variable_for_step = None
    best_hyperparameters_for_step = None

    # Iterate through remaining candidates to find the best one to add
    for candidate in remaining_candidates:
        current_variables = selected_variables + [candidate]
        print(f"  Evaluating adding: {candidate} with current set: {current_variables}")

        avg_mse_across_random_states = {} # To store avg MSE for each hyperparameter combo

        # Nested loop for random states
        for random_state in random_states:
            mse_list = []

            # Implement rolling window prediction
            for start_index in range(len(data_clean) - window_size - forecast_horizon + 1):
                end_index = start_index + window_size
                train_data = data_clean.iloc[start_index:end_index]
                test_data = data_clean.iloc[end_index:end_index + forecast_horizon]

                X_train = train_data[current_variables]
                y_train = train_data[target]
                X_test = test_data[current_variables]
                y_test = test_data[target]

                # Nested grid search for hyperparameters
                for params in ParameterGrid(param_grid):
                    # Train Random Forest model
                    model = RandomForestRegressor(random_state=random_state, **params)
                    model.fit(X_train, y_train)

                    # Predict and calculate MSE
                    predictions = model.predict(X_test)
                    mse = mean_squared_error(y_test, predictions)

                    # Store MSE for this hyperparameter combo and random state
                    if tuple(params.items()) not in avg_mse_across_random_states:
                         avg_mse_across_random_states[tuple(params.items())] = []
                    avg_mse_across_random_states[tuple(params.items())].append(mse)

        # Calculate average MSE across random states for each hyperparameter combo
        avg_mse_for_params = {
            params: np.mean(mse_list)
            for params, mse_list in avg_mse_across_random_states.items()
        }

        # Find the best hyperparameters for the current variable set
        best_params_for_candidate = min(avg_mse_for_params, key=avg_mse_for_params.get)
        min_avg_mse_for_candidate = avg_mse_for_params[best_params_for_candidate]

        # Check if this candidate resulted in the best MSE for this step
        if min_avg_mse_for_candidate < best_mse_for_step:
            best_mse_for_step = min_avg_mse_for_candidate
            best_variable_for_step = candidate
            best_hyperparameters_for_step = dict(best_params_for_candidate)


    # Add the best variable to the selected list and remove from candidates
    if best_variable_for_step:
        selected_variables.append(best_variable_for_step)
        remaining_candidates.remove(best_variable_for_step)
        print(f"\nSelected variable {i}: {best_variable_for_step}")
        print(f"Best MSE for this step: {best_mse_for_step}")
        print(f"Best Hyperparameters: {best_hyperparameters_for_step}")

        # Store the results for this step
        results_table.append({
            'Number of Variables': i,
            'Selected Variables': selected_variables.copy(),
            'Best n_estimators': best_hyperparameters_for_step.get('n_estimators'),
            'Best max_depth': best_hyperparameters_for_step.get('max_depth'),
            'Average Rolling Window MSE': best_mse_for_step
        })

        # Train and store the best model for this step (optional, depending on storage needs)
        # You might want to retrain the model on the full data_clean with the selected variables and best hyperparameters
        # best_model = RandomForestRegressor(random_state=random_state, **best_hyperparameters_for_step)
        # best_model.fit(data_clean[selected_variables], data_clean[target])
        # best_models[i] = best_model

    else:
        print("\nNo variable improved the MSE in this step. Stopping forward selection.")
        break

print("\nForward stepwise selection finished.")

# You can now display the results_table as a DataFrame
# display(pd.DataFrame(results_table))

Starting forward stepwise selection...

Selecting variable 1...
  Evaluating adding: pc1_lag1 with current set: ['pc1_lag1']
  Evaluating adding: pc1_lag2 with current set: ['pc1_lag2']
  Evaluating adding: pc1_lag3 with current set: ['pc1_lag3']
  Evaluating adding: pc1_lag4 with current set: ['pc1_lag4']
  Evaluating adding: pc3_lag1 with current set: ['pc3_lag1']
  Evaluating adding: pc3_lag2 with current set: ['pc3_lag2']
  Evaluating adding: pc3_lag3 with current set: ['pc3_lag3']
  Evaluating adding: pc3_lag4 with current set: ['pc3_lag4']
  Evaluating adding: dpc2_lag1 with current set: ['dpc2_lag1']
  Evaluating adding: dpc2_lag2 with current set: ['dpc2_lag2']
  Evaluating adding: dpc2_lag3 with current set: ['dpc2_lag3']
  Evaluating adding: dpc2_lag4 with current set: ['dpc2_lag4']
  Evaluating adding: dln_city_lag1 with current set: ['dln_city_lag1']
  Evaluating adding: dln_city_lag2 with current set: ['dln_city_lag2']
  Evaluating adding: dln_city_lag3 with current set: [

In [28]:
results_df1 = pd.DataFrame(results_table)
print("当前results_table的内容（已被XGBoost污染）:")
display(results_df1)

print("\n" + "="*50)
print("手动输入真实的RandomForest前向选择结果:")
print("="*50)

# 根据您之前截图显示的真实RandomForest前向选择结果手动输入
true_rf_results = [
    {
        'Number of Variables': 1,
        'Selected Variables': ['dln_gdp_sa_lag3'],
        'Best n_estimators': 150,
        'Best max_depth': 3,
        'Average Rolling Window MSE': 0.0005819163767493573
    },
    {
        'Number of Variables': 2,
        'Selected Variables': ['dln_gdp_sa_lag3', 'dln_gdp_sa_lag2'],
        'Best n_estimators': 75,
        'Best max_depth': 2,
        'Average Rolling Window MSE': 0.0004955223773497367
    },
    {
        'Number of Variables': 3,
        'Selected Variables': ['dln_gdp_sa_lag3', 'dln_gdp_sa_lag2', 'dln_region_lag2'],
        'Best n_estimators': 125,
        'Best max_depth': 2,
        'Average Rolling Window MSE': 0.0004283172860142044
    },
    {
        'Number of Variables': 4,
        'Selected Variables': ['dln_gdp_sa_lag3', 'dln_gdp_sa_lag2', 'dln_region_lag2', 'pc3_lag3'],
        'Best n_estimators': 125,
        'Best max_depth': 2,
        'Average Rolling Window MSE': 0.0004019424263143662
    },
    {
        'Number of Variables': 5,
        'Selected Variables': ['dln_gdp_sa_lag3', 'dln_gdp_sa_lag2', 'dln_region_lag2', 'pc3_lag3', 'dln_city_lag2'],
        'Best n_estimators': 125,
        'Best max_depth': 2,
        'Average Rolling Window MSE': 0.0003358649826809563
    },
    {
        'Number of Variables': 6,
        'Selected Variables': ['dln_gdp_sa_lag3', 'dln_gdp_sa_lag2', 'dln_region_lag2', 'pc3_lag3', 'dln_city_lag2', 'dpc2_lag3'],
        'Best n_estimators': 125,
        'Best max_depth': 2,
        'Average Rolling Window MSE': 0.0002723778988651875
    },
    {
        'Number of Variables': 7,
        'Selected Variables': ['dln_gdp_sa_lag3', 'dln_gdp_sa_lag2', 'dln_region_lag2', 'pc3_lag3', 'dln_city_lag2', 'dpc2_lag3', 'dpc2_lag1'],
        'Best n_estimators': 125,
        'Best max_depth': 5,
        'Average Rolling Window MSE': 0.0001666550123365229
    }
]

print("真实的RandomForest前向选择结果:")
true_rf_df = pd.DataFrame(true_rf_results)
display(true_rf_df)

当前results_table的内容（已被XGBoost污染）:


,Number of Variables,Selected Variables,Best n_estimators,Best max_depth,Average Rolling Window MSE
0,1,[dln_region_lag2],25,2,0.000499
1,2,"[dln_region_lag2, dln_gdp_sa_lag2]",25,2,0.000429
2,3,"[dln_region_lag2, dln_gdp_sa_lag2, pc1_lag4]",50,2,0.000324
3,4,"[dln_region_lag2, dln_gdp_sa_lag2, pc1_lag4, p...",75,2,0.000266
4,5,"[dln_region_lag2, dln_gdp_sa_lag2, pc1_lag4, p...",100,2,0.000250
5,6,"[dln_region_lag2, dln_gdp_sa_lag2, pc1_lag4, p...",75,2,0.000272
6,7,"[dln_region_lag2, dln_gdp_sa_lag2, pc1_lag4, p...",75,3,0.000291



手动输入真实的RandomForest前向选择结果:
真实的RandomForest前向选择结果:


,Number of Variables,Selected Variables,Best n_estimators,Best max_depth,Average Rolling Window MSE
0,1,[dln_gdp_sa_lag3],150,3,0.000582
1,2,"[dln_gdp_sa_lag3, dln_gdp_sa_lag2]",75,2,0.000496
2,3,"[dln_gdp_sa_lag3, dln_gdp_sa_lag2, dln_region_...",125,2,0.000428
3,4,"[dln_gdp_sa_lag3, dln_gdp_sa_lag2, dln_region_...",125,2,0.000402
4,5,"[dln_gdp_sa_lag3, dln_gdp_sa_lag2, dln_region_...",125,2,0.000336
5,6,"[dln_gdp_sa_lag3, dln_gdp_sa_lag2, dln_region_...",125,2,0.000272
6,7,"[dln_gdp_sa_lag3, dln_gdp_sa_lag2, dln_region_...",125,5,0.000167


In [11]:
# 在全部data_clean数据上训练前向选择的模型并保存
import joblib
import os

# 创建保存目录并训练模型
save_dir = r"D:\S_data\models"
os.makedirs(save_dir, exist_ok=True)
print(f"开始训练并保存{len(results_table)}个RandomForest模型...")

# 训练并保存所有模型
for i, result in enumerate(results_table, 1):
    # 获取变量和超参数
    vars_list = result['Selected Variables']
    n_est = result['Best n_estimators']
    best_dep = result['Best max_depth']
    num_vars = result['Number of Variables']
    
    # 训练模型
    model = RandomForestRegressor(n_estimators=n_est, max_depth=best_dep, random_state=10)
    model.fit(data_clean[vars_list], data_clean[target])
    
    # 按要求命名并保存模型：RF1, RF2, ..., RF7
    model_name = f"RF{num_vars}.pkl"
    model_path = os.path.join(save_dir, model_name)
    joblib.dump(model, model_path)
    
    print(f"已保存模型 {model_name} - 使用{len(vars_list)}个变量: {vars_list}")

print(f"\n完成！所有RandomForest模型已保存到: {save_dir}")
print("模型文件: RF1.pkl, RF2.pkl, RF3.pkl, RF4.pkl, RF5.pkl, RF6.pkl, RF7.pkl")

开始训练并保存7个RandomForest模型...
已保存模型 RF1.pkl - 使用1个变量: ['dln_gdp_sa_lag3']
已保存模型 RF2.pkl - 使用2个变量: ['dln_gdp_sa_lag3', 'dln_gdp_sa_lag2']
已保存模型 RF3.pkl - 使用3个变量: ['dln_gdp_sa_lag3', 'dln_gdp_sa_lag2', 'dln_region_lag2']
已保存模型 RF4.pkl - 使用4个变量: ['dln_gdp_sa_lag3', 'dln_gdp_sa_lag2', 'dln_region_lag2', 'pc3_lag3']
已保存模型 RF5.pkl - 使用5个变量: ['dln_gdp_sa_lag3', 'dln_gdp_sa_lag2', 'dln_region_lag2', 'pc3_lag3', 'dln_city_lag2']
已保存模型 RF6.pkl - 使用6个变量: ['dln_gdp_sa_lag3', 'dln_gdp_sa_lag2', 'dln_region_lag2', 'pc3_lag3', 'dln_city_lag2', 'dpc2_lag3']
已保存模型 RF7.pkl - 使用7个变量: ['dln_gdp_sa_lag3', 'dln_gdp_sa_lag2', 'dln_region_lag2', 'pc3_lag3', 'dln_city_lag2', 'dpc2_lag3', 'dpc2_lag1']

完成！所有RandomForest模型已保存到: D:\S_data\models
模型文件: RF1.pkl, RF2.pkl, RF3.pkl, RF4.pkl, RF5.pkl, RF6.pkl, RF7.pkl


In [30]:
# 创建详细的MSE结果表格 - 使用真实的RandomForest前向选择结果
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# 初始化结果列表
detailed_results = []

# 设置参数
window_size = 40
forecast_horizon = 5
random_states = [1, 2, 3]

# 使用手动输入的真实RandomForest前向选择结果（从之前截图读取）
true_rf_results = [
    {
        'Number of Variables': 1,
        'Selected Variables': ['dln_gdp_sa_lag3'],
        'Best n_estimators': 150,
        'Best max_depth': 3,
        'Average Rolling Window MSE': 0.0005819163767493573
    },
    {
        'Number of Variables': 2,
        'Selected Variables': ['dln_gdp_sa_lag3', 'dln_gdp_sa_lag2'],
        'Best n_estimators': 75,
        'Best max_depth': 2,
        'Average Rolling Window MSE': 0.0004955223773497367
    },
    {
        'Number of Variables': 3,
        'Selected Variables': ['dln_gdp_sa_lag3', 'dln_gdp_sa_lag2', 'dln_region_lag2'],
        'Best n_estimators': 125,
        'Best max_depth': 2,
        'Average Rolling Window MSE': 0.0004283172860142044
    },
    {
        'Number of Variables': 4,
        'Selected Variables': ['dln_gdp_sa_lag3', 'dln_gdp_sa_lag2', 'dln_region_lag2', 'pc3_lag3'],
        'Best n_estimators': 125,
        'Best max_depth': 2,
        'Average Rolling Window MSE': 0.0004019424263143662
    },
    {
        'Number of Variables': 5,
        'Selected Variables': ['dln_gdp_sa_lag3', 'dln_gdp_sa_lag2', 'dln_region_lag2', 'pc3_lag3', 'dln_city_lag2'],
        'Best n_estimators': 125,
        'Best max_depth': 2,
        'Average Rolling Window MSE': 0.0003358649826809563
    },
    {
        'Number of Variables': 6,
        'Selected Variables': ['dln_gdp_sa_lag3', 'dln_gdp_sa_lag2', 'dln_region_lag2', 'pc3_lag3', 'dln_city_lag2', 'dpc2_lag3'],
        'Best n_estimators': 125,
        'Best max_depth': 2,
        'Average Rolling Window MSE': 0.0002723778988651875
    },
    {
        'Number of Variables': 7,
        'Selected Variables': ['dln_gdp_sa_lag3', 'dln_gdp_sa_lag2', 'dln_region_lag2', 'pc3_lag3', 'dln_city_lag2', 'dpc2_lag3', 'dpc2_lag1'],
        'Best n_estimators': 125,
        'Best max_depth': 5,
        'Average Rolling Window MSE': 0.0001666550123365229
    }
]

print("使用手动输入的真实RandomForest前向选择结果计算详细MSE...")

# 从真实结果中获取数据并计算详细MSE
for step_result in true_rf_results:
    selected_vars = step_result['Selected Variables']
    num_vars = step_result['Number of Variables']
    best_n_est = step_result['Best n_estimators']
    best_depth = step_result['Best max_depth']
    original_mse = step_result['Average Rolling Window MSE']
    
    print(f"处理模型{num_vars}: {len(selected_vars)}个变量: {selected_vars}")
    print(f"  超参数: n_estimators={best_n_est}, max_depth={best_depth}")
    print(f"  原始MSE: {original_mse}")
    
    # 存储当前模型的所有MSE值
    all_mse_values = []
    mse_column_names = []
    
    # 滚动窗口预测
    for window_idx, start_index in enumerate(range(len(data_clean) - window_size - forecast_horizon + 1)):
        end_index = start_index + window_size
        train_data = data_clean.iloc[start_index:end_index]
        test_data = data_clean.iloc[end_index:end_index + forecast_horizon]
        
        X_train = train_data[selected_vars]
        y_train = train_data[target]
        X_test = test_data[selected_vars]
        y_test = test_data[target]
        
        # 对每个随机种子
        for random_state in random_states:
            # 训练模型 - 使用与前向选择完全相同的参数
            model = RandomForestRegressor(
                n_estimators=best_n_est, 
                max_depth=best_depth, 
                random_state=random_state
            )
            model.fit(X_train, y_train)
            
            # 预测并计算MSE
            predictions = model.predict(X_test)
            mse = mean_squared_error(y_test, predictions)
            all_mse_values.append(mse)
            
            # 创建列名：W1_R1, W1_R2, W1_R3, W2_R1, ...
            col_name = f'W{window_idx + 1}_R{random_state}'
            mse_column_names.append(col_name)
    
    # 计算平均MSE
    avg_mse = np.mean(all_mse_values)
    
    print(f"  重新计算的平均MSE: {avg_mse}")
    print(f"  与原始MSE的差异: {abs(avg_mse - original_mse)}")
    print()
    
    # 创建结果字典
    result_dict = {
        'Number of Variables': num_vars,
        'Selected Variables': ', '.join(selected_vars),
        'Average MSE': avg_mse,
        'Original Forward Selection MSE': original_mse,
        'MSE Difference': abs(avg_mse - original_mse)
    }
    
    # 添加每个具体的MSE值
    for j, mse_val in enumerate(all_mse_values):
        result_dict[mse_column_names[j]] = mse_val
    
    detailed_results.append(result_dict)

# 创建详细结果DataFrame
detailed_mse_df = pd.DataFrame(detailed_results)

print(f"\n详细MSE结果表格:")
print(f"表格形状: {detailed_mse_df.shape}")
print(f"包含列: Number of Variables, Selected Variables, Average MSE, Original Forward Selection MSE, MSE Difference, 以及 {len(all_mse_values)} 个详细MSE列")

# 显示表格（前几列）
print("\n前几列预览:")
display(detailed_mse_df.iloc[:, :7])  # 显示前7列

# 如果需要保存到Excel
save_path = r"D:\S_data\RF_all_MSE_TRUE.xlsx"
detailed_mse_df.to_excel(save_path, index=False)
print(f"\n✅ 详细MSE结果已保存到: {save_path}")

# 显示完整表格
print("\n完整表格:")
display(detailed_mse_df)

使用手动输入的真实RandomForest前向选择结果计算详细MSE...
处理模型1: 1个变量: ['dln_gdp_sa_lag3']
  超参数: n_estimators=150, max_depth=3
  原始MSE: 0.0005819163767493573
  重新计算的平均MSE: 0.0005819163767493572
  与原始MSE的差异: 1.0842021724855044e-19

处理模型2: 2个变量: ['dln_gdp_sa_lag3', 'dln_gdp_sa_lag2']
  超参数: n_estimators=75, max_depth=2
  原始MSE: 0.0004955223773497367
  重新计算的平均MSE: 0.0005819163767493572
  与原始MSE的差异: 1.0842021724855044e-19

处理模型2: 2个变量: ['dln_gdp_sa_lag3', 'dln_gdp_sa_lag2']
  超参数: n_estimators=75, max_depth=2
  原始MSE: 0.0004955223773497367
  重新计算的平均MSE: 0.0004955223773497367
  与原始MSE的差异: 0.0

处理模型3: 3个变量: ['dln_gdp_sa_lag3', 'dln_gdp_sa_lag2', 'dln_region_lag2']
  超参数: n_estimators=125, max_depth=2
  原始MSE: 0.0004283172860142044
  重新计算的平均MSE: 0.0004955223773497367
  与原始MSE的差异: 0.0

处理模型3: 3个变量: ['dln_gdp_sa_lag3', 'dln_gdp_sa_lag2', 'dln_region_lag2']
  超参数: n_estimators=125, max_depth=2
  原始MSE: 0.0004283172860142044
  重新计算的平均MSE: 0.00042831728601420445
  与原始MSE的差异: 5.421010862427522e-20

处理模型4: 4个变量: ['dln

,Number of Variables,Selected Variables,Average MSE,Original Forward Selection MSE,MSE Difference,W1_R1,W1_R2
0,1,dln_gdp_sa_lag3,0.000582,0.000582,1.084202e-19,0.000136,0.000201
1,2,"dln_gdp_sa_lag3, dln_gdp_sa_lag2",0.000496,0.000496,0.000000e+00,0.000874,0.000744
2,3,"dln_gdp_sa_lag3, dln_gdp_sa_lag2, dln_region_lag2",0.000428,0.000428,5.421011e-20,0.000429,0.000426
3,4,"dln_gdp_sa_lag3, dln_gdp_sa_lag2, dln_region_l...",0.000402,0.000402,0.000000e+00,0.000201,0.000184
4,5,"dln_gdp_sa_lag3, dln_gdp_sa_lag2, dln_region_l...",0.000336,0.000336,2.844129e-10,0.000670,0.000742
5,6,"dln_gdp_sa_lag3, dln_gdp_sa_lag2, dln_region_l...",0.000272,0.000272,1.084202e-19,0.000248,0.000235
6,7,"dln_gdp_sa_lag3, dln_gdp_sa_lag2, dln_region_l...",0.000167,0.000167,1.626303e-19,0.000301,0.000166



✅ 详细MSE结果已保存到: D:\S_data\RF_all_MSE_TRUE.xlsx

完整表格:


,Number of Variables,Selected Variables,Average MSE,Original Forward Selection MSE,MSE Difference,W1_R1,W1_R2,W1_R3,W2_R1,W2_R2,...,W7_R3,W8_R1,W8_R2,W8_R3,W9_R1,W9_R2,W9_R3,W10_R1,W10_R2,W10_R3
0,1,dln_gdp_sa_lag3,0.000582,0.000582,1.084202e-19,0.000136,0.000201,0.000253,0.000288,0.000242,...,0.000857,0.000352,0.000367,0.000386,0.000218,0.000242,0.000142,0.000107,0.000129,0.000092
1,2,"dln_gdp_sa_lag3, dln_gdp_sa_lag2",0.000496,0.000496,0.000000e+00,0.000874,0.000744,0.000770,0.000925,0.000665,...,0.000482,0.000202,0.000259,0.000379,0.000183,0.000261,0.000220,0.000364,0.000356,0.000309
2,3,"dln_gdp_sa_lag3, dln_gdp_sa_lag2, dln_region_lag2",0.000428,0.000428,5.421011e-20,0.000429,0.000426,0.000362,0.000304,0.000330,...,0.000632,0.000264,0.000227,0.000263,0.000152,0.000147,0.000158,0.000245,0.000181,0.000236
3,4,"dln_gdp_sa_lag3, dln_gdp_sa_lag2, dln_region_l...",0.000402,0.000402,0.000000e+00,0.000201,0.000184,0.000412,0.000215,0.000132,...,0.000604,0.000277,0.000258,0.000305,0.000210,0.000268,0.000199,0.000238,0.000263,0.000328
4,5,"dln_gdp_sa_lag3, dln_gdp_sa_lag2, dln_region_l...",0.000336,0.000336,2.844129e-10,0.000670,0.000742,0.000631,0.000935,0.000876,...,0.000386,0.000175,0.000085,0.000169,0.000041,0.000075,0.000072,0.000079,0.000063,0.000122
5,6,"dln_gdp_sa_lag3, dln_gdp_sa_lag2, dln_region_l...",0.000272,0.000272,1.084202e-19,0.000248,0.000235,0.000277,0.000398,0.000547,...,0.000130,0.000155,0.000134,0.000157,0.000457,0.000333,0.000450,0.000396,0.000265,0.000210
6,7,"dln_gdp_sa_lag3, dln_gdp_sa_lag2, dln_region_l...",0.000167,0.000167,1.626303e-19,0.000301,0.000166,0.000443,0.000381,0.000559,...,0.000085,0.000063,0.000066,0.000054,0.000233,0.000132,0.000134,0.000082,0.000045,0.000060


# XGboost

In [13]:
# Define the hyperparameter grid for XGBoost
param_grid = {
    'n_estimators': [25, 50, 75, 100],
    'max_depth': [1, 2, 3]
}

# Define the range of random states for averaging
random_states = range(1, 4)

# Define fixed parameters for XGBoost
fixed_params = {
    'subsample': 0.95,
    'colsample_bytree': 1,
    'learning_rate': 0.05,
    'objective': 'reg:squarederror', # Objective for regression
    'seed': 10 # Fixed seed for reproducibility within XGBoost
}


print("XGBoost Hyperparameter Grid:", param_grid)
print("Random States for averaging:", list(random_states))
print("Fixed XGBoost Parameters:", fixed_params)

XGBoost Hyperparameter Grid: {'n_estimators': [25, 50, 75, 100], 'max_depth': [1, 2, 3]}
Random States for averaging: [1, 2, 3]
Fixed XGBoost Parameters: {'subsample': 0.95, 'colsample_bytree': 1, 'learning_rate': 0.05, 'objective': 'reg:squarederror', 'seed': 10}


In [14]:
from xgboost import XGBRegressor
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd # Import pandas for DataFrame creation

# Initialize variables for forward selection
selected_variables = []
best_models = {}
results_table = [] # To store results for the final table

max_variables = 7
window_size = 40
forecast_horizon = 5

# Ensure data_clean is sorted by time, as required for time series
data_clean = data_clean.sort_values(by='time').reset_index(drop=True)

# Define the possible candidate variables that haven't been selected yet
remaining_candidates = list(candidates)

print("Starting forward stepwise selection with XGBoost...")

# Forward stepwise selection loop
for i in range(1, max_variables + 1):
    print(f"\nSelecting variable {i}...")
    best_mse_for_step = float('inf')
    best_variable_for_step = None
    best_hyperparameters_for_step = None

    # Iterate through remaining candidates to find the best one to add
    for candidate in remaining_candidates:
        current_variables = selected_variables + [candidate]
        # print(f"  Evaluating adding: {candidate} with current set: {current_variables}") # Optional: uncomment for detailed progress

        avg_mse_across_random_states_per_params = {} # To store avg MSE for each hyperparameter combo across random states

        # Nested loop for random states and rolling window
        for random_state in random_states:
             # Implement rolling window prediction
            for start_index in range(len(data_clean) - window_size - forecast_horizon + 1):
                end_index = start_index + window_size
                train_data = data_clean.iloc[start_index:end_index]
                test_data = data_clean.iloc[end_index:end_index + forecast_horizon]

                X_train = train_data[current_variables]
                y_train = train_data[target]
                X_test = test_data[current_variables]
                y_test = test_data[target]

                # Nested grid search for hyperparameters
                for params in ParameterGrid(param_grid):
                    # Train XGBoost model
                    # Use the random_state from the outer loop for averaging
                    # Exclude 'seed' from fixed_params when passing to the constructor
                    model = XGBRegressor(random_state=random_state, **params, **{k: v for k, v in fixed_params.items() if k != 'seed'})
                    model.fit(X_train, y_train)

                    # Predict and calculate MSE
                    predictions = model.predict(X_test)
                    mse = mean_squared_error(y_test, predictions)

                    # Store MSE for this hyperparameter combo and random state
                    param_key = tuple(sorted(params.items())) # Use sorted items for consistent key
                    if param_key not in avg_mse_across_random_states_per_params:
                         avg_mse_across_random_states_per_params[param_key] = []
                    avg_mse_across_random_states_per_params[param_key].append(mse)


        # Calculate overall average MSE across random states and rolling windows for each hyperparameter combo
        overall_avg_mse_for_params = {
            params_key: np.mean(mse_list)
            for params_key, mse_list in avg_mse_across_random_states_per_params.items()
        }


        # Find the best hyperparameters for the current variable set based on overall average MSE
        if overall_avg_mse_for_params: # Check if dictionary is not empty
            best_params_for_candidate_key = min(overall_avg_mse_for_params, key=overall_avg_mse_for_params.get)
            min_avg_mse_for_candidate = overall_avg_mse_for_params[best_params_for_candidate_key]
            best_params_for_candidate = dict(best_params_for_candidate_key)

            # Check if this candidate resulted in the best MSE for this step
            if min_avg_mse_for_candidate < best_mse_for_step:
                best_mse_for_step = min_avg_mse_for_candidate
                best_variable_for_step = candidate
                best_hyperparameters_for_step = best_params_for_candidate
        else:
             print(f"Warning: No MSE values calculated for candidate {candidate}. Skipping.")


    # Add the best variable to the selected list and remove from candidates
    if best_variable_for_step:
        selected_variables.append(best_variable_for_step)
        remaining_candidates.remove(best_variable_for_step)
        print(f"\nSelected variable {i}: {best_variable_for_step}")
        print(f"Best MSE for this step: {best_mse_for_step}")
        print(f"Best Hyperparameters: {best_hyperparameters_for_step}")

        # Store the results for this step
        results_table.append({
            'Number of Variables': i,
            'Selected Variables': selected_variables.copy(),
            'Best n_estimators': best_hyperparameters_for_step.get('n_estimators'),
            'Best max_depth': best_hyperparameters_for_step.get('max_depth'),
            'Average Rolling Window MSE': best_mse_for_step
        })

    else:
        print("\nNo variable improved the MSE in this step. Stopping forward selection.")
        break

print("\nForward stepwise selection with XGBoost finished.")

Starting forward stepwise selection with XGBoost...

Selecting variable 1...

Selected variable 1: dln_region_lag2
Best MSE for this step: 0.000499301749595525
Best Hyperparameters: {'max_depth': 2, 'n_estimators': 25}

Selecting variable 2...

Selected variable 2: dln_gdp_sa_lag2
Best MSE for this step: 0.0004289541825265397
Best Hyperparameters: {'max_depth': 2, 'n_estimators': 25}

Selecting variable 3...

Selected variable 3: pc1_lag4
Best MSE for this step: 0.00032420178800731644
Best Hyperparameters: {'max_depth': 2, 'n_estimators': 50}

Selecting variable 4...

Selected variable 4: pc3_lag3
Best MSE for this step: 0.00026637394066105017
Best Hyperparameters: {'max_depth': 2, 'n_estimators': 75}

Selecting variable 5...

Selected variable 5: pc3_lag1
Best MSE for this step: 0.00025001549246975944
Best Hyperparameters: {'max_depth': 2, 'n_estimators': 100}

Selecting variable 6...

Selected variable 6: dln_gdp_sa_lag3
Best MSE for this step: 0.00027209281122894056
Best Hyperparame

In [15]:
results_df2 = pd.DataFrame(results_table)
display(results_df2)

,Number of Variables,Selected Variables,Best n_estimators,Best max_depth,Average Rolling Window MSE
0,1,[dln_region_lag2],25,2,0.000499
1,2,"[dln_region_lag2, dln_gdp_sa_lag2]",25,2,0.000429
2,3,"[dln_region_lag2, dln_gdp_sa_lag2, pc1_lag4]",50,2,0.000324
3,4,"[dln_region_lag2, dln_gdp_sa_lag2, pc1_lag4, p...",75,2,0.000266
4,5,"[dln_region_lag2, dln_gdp_sa_lag2, pc1_lag4, p...",100,2,0.000250
5,6,"[dln_region_lag2, dln_gdp_sa_lag2, pc1_lag4, p...",75,2,0.000272
6,7,"[dln_region_lag2, dln_gdp_sa_lag2, pc1_lag4, p...",75,3,0.000291


In [16]:
# 在全部data_clean数据上训练前向选择的模型并保存
import joblib
import os

# 创建保存目录并训练模型
save_dir = r"D:\S_data\models"
os.makedirs(save_dir, exist_ok=True)
print(f"开始训练并保存{len(results_table)}个xgboost模型...")

# 训练并保存所有模型
for i, result in enumerate(results_table, 1):
    # 获取变量和超参数
    vars_list = result['Selected Variables']
    n_est = result['Best n_estimators']
    best_dep = result['Best max_depth']
    num_vars = result['Number of Variables']
    
    # 训练模型
    model = XGBRegressor(n_estimators=n_est, max_depth=best_dep, random_state=10)
    model.fit(data_clean[vars_list], data_clean[target])
    
    # 按要求命名并保存模型
    model_name = f"xgb{num_vars}.pkl"
    model_path = os.path.join(save_dir, model_name)
    joblib.dump(model, model_path)
    
    print(f"已保存模型 {model_name} - 使用{len(vars_list)}个变量: {vars_list}")

print(f"\n完成！所有RandomForest模型已保存到: {save_dir}")
print("模型文件: xgb1.pkl, xgb2.pkl, xgb3.pkl, xgb4.pkl, RF5.pkl, RF6.pkl, RF7.pkl")

开始训练并保存7个xgboost模型...
已保存模型 xgb1.pkl - 使用1个变量: ['dln_region_lag2']
已保存模型 xgb2.pkl - 使用2个变量: ['dln_region_lag2', 'dln_gdp_sa_lag2']
已保存模型 xgb3.pkl - 使用3个变量: ['dln_region_lag2', 'dln_gdp_sa_lag2', 'pc1_lag4']
已保存模型 xgb4.pkl - 使用4个变量: ['dln_region_lag2', 'dln_gdp_sa_lag2', 'pc1_lag4', 'pc3_lag3']
已保存模型 xgb5.pkl - 使用5个变量: ['dln_region_lag2', 'dln_gdp_sa_lag2', 'pc1_lag4', 'pc3_lag3', 'pc3_lag1']
已保存模型 xgb6.pkl - 使用6个变量: ['dln_region_lag2', 'dln_gdp_sa_lag2', 'pc1_lag4', 'pc3_lag3', 'pc3_lag1', 'dln_gdp_sa_lag3']
已保存模型 xgb7.pkl - 使用7个变量: ['dln_region_lag2', 'dln_gdp_sa_lag2', 'pc1_lag4', 'pc3_lag3', 'pc3_lag1', 'dln_gdp_sa_lag3', 'dpc2_lag1']

完成！所有RandomForest模型已保存到: D:\S_data\models
模型文件: xgb1.pkl, xgb2.pkl, xgb3.pkl, xgb4.pkl, RF5.pkl, RF6.pkl, RF7.pkl


In [25]:
# 创建XGBoost详细MSE结果表格
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

detailed_results = []
window_size, forecast_horizon, random_states = 40, 5, [1, 2, 3]

# 定义与前向选择时相同的固定参数
fixed_params = {
    'subsample': 0.95,
    'colsample_bytree': 1,
    'learning_rate': 0.05,
    'objective': 'reg:squarederror'
}

# 从XGBoost的results_table获取数据
for step_result in results_table:
    selected_vars, num_vars = step_result['Selected Variables'], step_result['Number of Variables']
    best_n_est, best_depth = step_result['Best n_estimators'], step_result['Best max_depth']
    
    all_mse_values, mse_column_names = [], []
    
    # 滚动窗口预测
    for window_idx, start_index in enumerate(range(len(data_clean) - window_size - forecast_horizon + 1)):
        end_index = start_index + window_size
        X_train, y_train = data_clean.iloc[start_index:end_index][selected_vars], data_clean.iloc[start_index:end_index][target]
        X_test, y_test = data_clean.iloc[end_index:end_index + forecast_horizon][selected_vars], data_clean.iloc[end_index:end_index + forecast_horizon][target]
        
        for random_state in random_states:
            # 使用与前向选择时完全相同的参数配置
            model = XGBRegressor(
                n_estimators=best_n_est, 
                max_depth=best_depth, 
                random_state=random_state,
                **fixed_params
            )
            model.fit(X_train, y_train)
            mse = mean_squared_error(y_test, model.predict(X_test))
            all_mse_values.append(mse)
            mse_column_names.append(f'W{window_idx + 1}_R{random_state}')
    
    result_dict = {'Number of Variables': num_vars, 'Selected Variables': ', '.join(selected_vars), 'Average MSE': np.mean(all_mse_values)}
    result_dict.update({mse_column_names[i]: mse_val for i, mse_val in enumerate(all_mse_values)})
    detailed_results.append(result_dict)

# 创建并保存结果
xgb_detailed_df = pd.DataFrame(detailed_results)
xgb_detailed_df.to_excel(r"D:\S_data\XGB_all_MSE.xlsx", index=False)
print(f"XGBoost详细MSE表格形状: {xgb_detailed_df.shape}")
print("现在使用与前向选择过程完全相同的参数配置:")
print(f"固定参数: {fixed_params}")
display(xgb_detailed_df)

XGBoost详细MSE表格形状: (7, 33)
现在使用与前向选择过程完全相同的参数配置:
固定参数: {'subsample': 0.95, 'colsample_bytree': 1, 'learning_rate': 0.05, 'objective': 'reg:squarederror'}


,Number of Variables,Selected Variables,Average MSE,W1_R1,W1_R2,W1_R3,W2_R1,W2_R2,W2_R3,W3_R1,...,W7_R3,W8_R1,W8_R2,W8_R3,W9_R1,W9_R2,W9_R3,W10_R1,W10_R2,W10_R3
0,1,dln_region_lag2,0.000499,0.000265,0.000248,0.000253,0.000271,0.000239,0.000244,0.000829,...,0.000858,0.000212,0.000204,0.000217,0.000047,0.000050,0.000057,0.000057,0.000065,0.000048
1,2,"dln_region_lag2, dln_gdp_sa_lag2",0.000429,0.000103,0.000082,0.000157,0.000365,0.000270,0.000315,0.000583,...,0.000796,0.000201,0.000217,0.000251,0.000317,0.000258,0.000208,0.000268,0.000327,0.000247
2,3,"dln_region_lag2, dln_gdp_sa_lag2, pc1_lag4",0.000324,0.000183,0.000163,0.000204,0.000382,0.000432,0.000226,0.000531,...,0.000598,0.000132,0.000126,0.000176,0.000050,0.000052,0.000055,0.000121,0.000164,0.000101
3,4,"dln_region_lag2, dln_gdp_sa_lag2, pc1_lag4, pc...",0.000266,0.000260,0.000253,0.000170,0.000301,0.000302,0.000248,0.000173,...,0.000359,0.000171,0.000182,0.000215,0.000163,0.000100,0.000101,0.000124,0.000175,0.000161
4,5,"dln_region_lag2, dln_gdp_sa_lag2, pc1_lag4, pc...",0.000250,0.000144,0.000264,0.000237,0.000179,0.000186,0.000310,0.000304,...,0.000387,0.000217,0.000272,0.000213,0.000153,0.000128,0.000192,0.000165,0.000345,0.000195
5,6,"dln_region_lag2, dln_gdp_sa_lag2, pc1_lag4, pc...",0.000272,0.000201,0.000264,0.000204,0.000239,0.000241,0.000294,0.000249,...,0.000384,0.000188,0.000277,0.000166,0.000190,0.000159,0.000129,0.000076,0.000135,0.000163
6,7,"dln_region_lag2, dln_gdp_sa_lag2, pc1_lag4, pc...",0.000291,0.000060,0.000140,0.000189,0.000072,0.000108,0.000148,0.000130,...,0.000471,0.000160,0.000208,0.000157,0.000192,0.000225,0.000198,0.000529,0.000324,0.000323
